In [1]:
import time
from datetime import datetime

import akshare as ak


In [ ]:
"""
version 1:

参数化
统计连续复合上引线的最大时间段

获取N天的行情、统计每天的上引线情况，复合条件为True
然后获取N天内上引线情况？

每个票的板块、概念信息
存redis吧

如何自动适配参数！
"""

In [16]:
import akshare as ak
import json
import pandas as pd
from datetime import datetime

data = {}

df_list = []

stock_board_concept_name_ths_df = ak.stock_board_concept_name_ths()

for idx, row in stock_board_concept_name_ths_df.iterrows():
    # key: 日期、概念名称、成分股数量、网址、代码
    row_dct = row.to_dict()
    cc_code = row_dct['代码']
    concept = row_dct['概念名称']
    
    stock_board_cons_ths_df = ak.stock_board_cons_ths(symbol=cc_code)
    stock_board_cons_ths_df['概念'] = concept
    stock_board_cons_ths_df = stock_board_cons_ths_df.drop('序号', axis=1)
    df_list.append(stock_board_cons_ths_df)
    time.sleep(5)

ndf = pd.concat(df_list)

def combine(ser):
    if ser.name == '概念':
        return ','.join(ser)
    return ser.tolist()[0]

# ndf = ndf.groupby('代码').aggregate(combine)
# ndf.to_csv('concept.csv')

In [17]:
import akshare as ak


def get_trade(symbol):
    """
    获取行业信息
    :param symbol:
    :return:
    """
    df = ak.stock_individual_info_em(symbol=symbol)
    dct = dict(zip(df['item'], df['value']))
    return dct['行业'], dct['股票简称']

get_trade('003004')

('计算机设备', '声迅股份')

In [15]:
from pyecharts import options as opts
from pyecharts.globals import CurrentConfig, OnlineHostType
from pyecharts.charts import Kline, Line, Grid, Bar
from pyecharts.globals import CurrentConfig, NotebookType
from IPython.display import display
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST
import pandas as pd
import os

data = [
    [2320.26, 2320.26, 2287.3, 2362.94],
    [2300, 2291.3, 2288.26, 2308.38],
    [2295.35, 2346.5, 2295.35, 2345.92],
    [2347.22, 2358.98, 2337.35, 2363.8],
    [2360.75, 2382.48, 2347.89, 2383.76],
]

def plot_kline(df, symbol):
    date_list = df['日期'].to_list()
    ndf = df[['开盘', '收盘', '最低', '最高']]
    data = ndf.values.tolist()
    volume = df['成交量'].tolist()
    
    area, name = get_trade(symbol)
    
    kline = (
    Kline()
    .add_xaxis(date_list)
    .add_yaxis(name, data)
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        xaxis_opts=opts.AxisOpts(is_scale=True),
        # datazoom_opts=[opts.DataZoomOpts(type_='slider')],
        datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=False,
                    type_="inside",
                    xaxis_index=[0,1],  # 这里需要修改可缩放的x轴坐标编号
                    range_start=60,
                    range_end=100,
                ),
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=[0,1],  # 这里需要修改可缩放的x轴坐标编号
                    type_="slider",
                    pos_top="94%",
                    range_start=60,
                    range_end=100,
                ),
        ],

    ))
    
    line = (
        Line()
        .add_xaxis(xaxis_data=date_list)
        .add_yaxis(
            series_name="MA5",
            y_axis=df['ma_5'].tolist(),
            is_smooth=True,
            is_symbol_show=False, # 是否显示小圆点
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=1.5, opacity=0.8, color="#4B4453"),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA10",
            y_axis=df['ma_10'].tolist(),
            is_smooth=True,
            is_symbol_show=False, # 是否显示小圆点
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=1.5, opacity=0.8, color="#FF8C00"),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA20",
            y_axis=df['ma_20'].tolist(),
            is_smooth=True,
            is_symbol_show=False, # 是否显示小圆点
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=1.5, opacity=0.8, color="#4B0082"),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"),
                        yaxis_opts=opts.AxisOpts(
                        is_scale=True,
                        splitarea_opts=opts.SplitAreaOpts(is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1))
                        )
        )
    )
    
    
    bar = (
        Bar()
        .add_xaxis(xaxis_data=date_list)
        .add_yaxis(
            series_name="volume",
            y_axis=volume,
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
            itemstyle_opts=opts.ItemStyleOpts(),
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category",
                grid_index=1,
                axislabel_opts=opts.LabelOpts(is_show=False),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )

    
    # Kline And Line
    overlap_kline_line = kline.overlap(line)

    # Grid Overlap + Bar
    grid_chart = Grid(
        init_opts=opts.InitOpts(
            width="980px",
            height="550px",
            animation_opts=opts.AnimationOpts(animation=False),
        )
    )
    grid_chart.add(
        overlap_kline_line,
        grid_opts=opts.GridOpts(pos_left="12%", pos_right="12%", height="55%")
    )
    
    grid_chart.add(
        bar,
        grid_opts=opts.GridOpts(
            pos_left="12%", pos_right="12%", pos_top="75%", height="15%"
        ),
    )


    grid_chart.load_javascript()
    time.sleep(1)
    return grid_chart.render_notebook()
    
import time

folder = 'daily_data/'

files = os.listdir(folder)
df = pd.read_csv(os.path.join(folder, '003004.csv'))
display(plot_kline(df, '003004'))

### 频繁上影线异动筛选

In [ ]:
import pandas as pd

ndf = pd.read_csv('concept.csv')

In [ ]:
import os

count = 0
times_of_unusual = 5
folder = 'daily_data/'

files = os.listdir(folder)

for f in files:
    # try:
        if not f.endswith('.csv'):
            continue
            
        code = f.split('.')[0]
        df = pd.read_csv(os.path.join(folder, f), encoding='utf-8')
        
        # 10日均线在20日均线上、最新股价在60日均线上
        # 不能偏离60日线太远
        last_row_data = df.iloc[-1].to_dict()
        last_cond1 = last_row_data['ma_10'] >= last_row_data['ma_20']*0.98
        last_cond2 = last_row_data['收盘'] >= last_row_data['ma_60']*0.98
        last_cond3 = (last_row_data['收盘'] - last_row_data['ma_60'])/last_row_data['收盘']*100 < 20
        if not (last_cond1 and last_cond2 and last_cond3):
            continue

        # 条件筛选
        # 换手率大于 2.5, 小于25 【真实的换手率其实需要排除掉一些实控人的资金，再计算的，这里不算严格】
        cond1 = (df['换手率'] > 1.5) & (df['换手率'] < 25)

        # 振幅大于 6.5
        cond3 = df['振幅'] > 6.5

        # (-2 <= 涨跌幅 <= 5.5) [收在-2到5.5之间]
        # cond4 = (-2 < df['涨跌幅']) & (df['涨跌幅'] <= 5.5)

        # 上影线是实体的0.8倍以上 【放宽条件，后续进行人工筛选】
        # cond5 = df['upper_lead'] >= df['len_of_entity'] * 0.8
        cond5 = df['upper_lead'] >= 2.8

        # 下引线小于2.3
        # cond6 = df['lower_lead'] < 2.5
        
        # 连续3个交易日的换手率
        df['ex_ratio_3'] = df['换手率'].rolling(3).sum()
        df = df.fillna(0)
        if any((df['ex_ratio_3'] >= 60)):
            # print(code, df['换手率'].mean(), max(df['ex_ratio_3']))
            # 三天换手率就超过了60，视为基本出货完毕
            continue
        
        # 连续21天上涨超过32
        df['up_down'] = df['涨跌幅'].rolling(21).sum()
        df = df.fillna(0)
        if any((df['up_down'] >= 32)):
            # print(code, max(df['up_down']))
            continue
        

        df['flag'] = cond1 & cond3 & cond5
        sdf = df[df['flag']]
        if not sdf.empty and len(sdf) >= 6:
            first_row = ndf[ndf['代码'].isin([code])].iloc[0]
            first_dct = first_row.to_dict()
            
            if 'st' in first_dct['名称']:
                continue
            
            print(code, first_dct['名称'], f'流通市值：{first_dct["流通市值"]}')
            
            print(df['换手率'].mean(), max(df['ex_ratio_3']))
            
            print('距离60日线点位:', (last_row_data['收盘'] - last_row_data['ma_60'])/last_row_data['收盘']*100)
            print(f'异动次数：{len(sdf)}')
            print(first_dct['概念'])
            # print('\n'.join(sdf['日期']))
            display(plot_kline(df, code))
            print('-'*10)

            count += 1

        # if count > 1000:
            # break
    # except Exception as e:
    #     print(f)
    #     print(e)


print(count)
"""
股价新高的不要，欧晶科技
反例：
宝明科技
"""

In [ ]:
# 连续几天超高换手率的过滤、或者连续大涨过的   中通国脉 
# 股价维持在高位的不要   丽江股份
# 股价没有在某段时间连续大涨 鱼跃医疗 
# 上引线是高点，jsqj

# 均价维持高位，不考虑
# 统计横盘情况？


# 中衡设计
# 换手率要用相对